In [ ]:
c%%bash
pip install transformers
pip install demoji
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 635.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import warnings
warnings.filterwarnings('ignore')

import demoji
import contractions
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# Preprocessing

1.   Remove emojis
2.   Remove extra whitespace
3.   Change contractions

No lowercasing and stopword removal as they were deteriorating the output quality as observed by sampling a few reviews.

Future work: Microtext normalisation

In [ ]:
def remove_emojis(review):
    return demoji.replace(review, '')

In [ ]:
def remove_extra_whitespace(review):
    lines = []
    for line in review.splitlines():
        if line == '' or line == ' ':
            continue
        lines.append(' '.join(line.split()))
    return '\n'.join(lines)

In [ ]:
def change_contractions(review):
    lines = []
    for line in review.splitlines():
        expanded_words = [contractions.fix(word) for word in line.split()]
        lines.append(' '.join(expanded_words))
    return '\n'.join(lines)

In [ ]:
df = pd.read_csv('drive/MyDrive/SC4021/test.csv')
df

,Unnamed: 0,title,stars,text,publishedAtDate,likesCount,name,reviewerNumberOfReviews,isLocalGuide,responseFromOwnerDate,...,reviewImageUrls/42,reviewImageUrls/43,reviewImageUrls/44,reviewImageUrls/45,reviewImageUrls/46,reviewImageUrls/47,reviewImageUrls/48,reviewImageUrls/49,lang,num_words
0,7678,Iro Iro Japanese Restaurant,5,We were so excited to eat so the photos are ta...,2023-11-25T02:55:12.562Z,1,Lee Joleen,39.0,False,2023-12-10T15:15:19.003Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,338
1,5117,Pura Brasa Singapore,5,Really great Spanish food at reasonable prices...,2024-02-14T16:50:26.656Z,0,Randy Choo,26.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,40
2,3138,Sheik Jalani Restaurant,4,"Food range is amazing, food equality is slight...",2018-04-19T05:10:55.527Z,0,Miss Ain M,61.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,32
3,2683,Black Society 黑社会,5,Nice ambience - the room was well decorated wi...,2023-12-23T12:36:58.020Z,0,Marilyn Lau,2.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,46
4,620,BOMBAY DINING-NORTH INDIAN RESTAURANT,2,Ordered on Deliveroo and food arrived 50min la...,2021-05-22T11:13:00.211Z,4,Owen Lam,47.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,6431,Melben Seafood Restaurant - Depot Lane,2,Purchased a mongolian style beef rice with egg...,2023-11-25T15:04:57.753Z,0,jinsen goh,48.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,26
1064,2572,Tambi Restaurant,5,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,2024-02-21T13:37:15.730Z,0,Layla Said,37.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,57
1065,9377,Ba Shu Sichuan Restaurant,5,Great tasting and authentic dishes here.\n\nWe...,2022-01-31T04:32:26.426Z,5,Jonneh Ng,8.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,58
1066,9498,Mendon Bedok Mall,5,a small cozy nook beside Tori-Q that serves re...,2023-11-05T10:49:49.070Z,0,Xing Yi Han,5.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,90


In [ ]:
df = df[['title', 'stars', 'text']]
df

,title,stars,text
0,Iro Iro Japanese Restaurant,5,We were so excited to eat so the photos are ta...
1,Pura Brasa Singapore,5,Really great Spanish food at reasonable prices...
2,Sheik Jalani Restaurant,4,"Food range is amazing, food equality is slight..."
3,Black Society 黑社会,5,Nice ambience - the room was well decorated wi...
4,BOMBAY DINING-NORTH INDIAN RESTAURANT,2,Ordered on Deliveroo and food arrived 50min la...
...,...,...,...
1063,Melben Seafood Restaurant - Depot Lane,2,Purchased a mongolian style beef rice with egg...
1064,Tambi Restaurant,5,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...
1065,Ba Shu Sichuan Restaurant,5,Great tasting and authentic dishes here.\n\nWe...
1066,Mendon Bedok Mall,5,a small cozy nook beside Tori-Q that serves re...


In [ ]:
cleaned_reviews = []
n = df.shape[0]

for i in range(n):
    review = df['text'][i]

    review = remove_emojis(review)
    review = remove_extra_whitespace(review)
    review = change_contractions(review)

    cleaned_reviews.append(review)

df['cleaned_reviews'] = cleaned_reviews
df

,title,stars,text,cleaned_reviews
0,Iro Iro Japanese Restaurant,5,We were so excited to eat so the photos are ta...,We were so excited to eat so the photos are ta...
1,Pura Brasa Singapore,5,Really great Spanish food at reasonable prices...,Really great Spanish food at reasonable prices...
2,Sheik Jalani Restaurant,4,"Food range is amazing, food equality is slight...","Food range is amazing, food equality is slight..."
3,Black Society 黑社会,5,Nice ambience - the room was well decorated wi...,Nice ambience - the room was well decorated wi...
4,BOMBAY DINING-NORTH INDIAN RESTAURANT,2,Ordered on Deliveroo and food arrived 50min la...,Ordered on Deliveroo and food arrived 50min la...
...,...,...,...,...
1063,Melben Seafood Restaurant - Depot Lane,2,Purchased a mongolian style beef rice with egg...,Purchased a mongolian style beef rice with egg...
1064,Tambi Restaurant,5,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...
1065,Ba Shu Sichuan Restaurant,5,Great tasting and authentic dishes here.\n\nWe...,Great tasting and authentic dishes here.\nWe o...
1066,Mendon Bedok Mall,5,a small cozy nook beside Tori-Q that serves re...,a small cozy nook beside Tori-Q that serves re...


### Sampling cleaned reviews

In [ ]:
print(df['text'][777])
print(df['cleaned_reviews'][777])

wow, i had a really enjoyable dinner!  the yuzu-themed dishes were very good!  the yuzu soup and waygu were very yummy!

came here for dinner during less busy weekday.  ordered one Signature Yuzu Shabu Set with A5 Hokkaido Snow Wagyu and one Ishiyaki Hot Stone Grill (yakiniku) Set with A4 Wagyu.

the service was good, and the ambience was good & quiet.  the generous spacing of seats and the tall ceiling made the environment felt very comfortable.  easy parking right outside.

a hidden gem in this area!!
wow, i had a really enjoyable dinner! the yuzu-themed dishes were very good! the yuzu soup and waygu were very yummy!
came here for dinner during less busy weekday. ordered one Signature Yuzu Shabu Set with A5 Hokkaido Snow Wagyu and one Ishiyaki Hot Stone Grill (yakiniku) Set with A4 Wagyu.
the service was good, and the ambience was good & quiet. the generous spacing of seats and the tall ceiling made the environment felt very comfortable. easy parking right outside.
a hidden gem in th

In [ ]:
print(df['text'][80])
print(df['cleaned_reviews'][80])

Tried a new restaurant close-by the CBD. Had the set lunch, and the food was not bad. My favourite item was the Croûte aux Champignons 🤤

Pictured:
- Croûte aux Champignons: Wild Mushrooms Ragout on Toasted Bread
- Mignons de Porc au Poivre Noir: Pork Tenderloin, Black Pepper Sauce & Potato Rösti
- Filet de Bar, Sauce Vierge: Sea Bass Fillet, Virgin Sauce & Seasonal Vegetables
- Coupe Romanoff: Vanilla Ice-Cream, fresh Strawberry and Chantilly Cream
Tried a new restaurant close-by the CBD. Had the set lunch, and the food was not bad. My favourite item was the Croûte aux Champignons
Pictured:
- Croûte aux Champignons: Wild Mushrooms Ragout on Toasted Bread
- Mignons de Porc au Poivre Noir: Pork Tenderloin, Black Pepper Sauce & Potato Rösti
- Filet de Bar, Sauce Vierge: Sea Bass Fillet, Virgin Sauce & Seasonal Vegetables
- Coupe Romanoff: Vanilla Ice-Cream, fresh Strawberry and Chantilly Cream


# Aspect Based Sentiment Analysis (ABSA)

*   Approach: InstructABSA
*   Model: Tk-Instruct instruction-tuned for 6 ABSA tasks
*   Relevant task: Aspect-Sentiment Pair Extraction (ASPE)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants")
model = AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants")

bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm: none.
    Positive example 1-
    input: Great spiced apple drink.
    output: spiced apple drink: positive
    Positive example 2-
    input: A very cozy place with great service and attention. Food is great and ambience are good.
    output: place: positive, service: positive, food: positive, ambience: positive
    Negative example 1-
    input: The small pizza is tiny and up the road you can get a 3 course meal for 15 soles!
    output: small pizza: negative
    Negative example 2-
    input: Mushroom watery and lemon tea need to ask for extra lemon.
    output: Mushroom: negative, lemon tea: negative
    Neutral example 1-
    input: Food was acceptable.
    output: Food: neutral
    Neutral example 2-
    input: Thai Iced Tea and potstickers were meh.
    output: Thai Iced Tea: neutral, potstickers: neutral
    Now complete the following example-
    input: """
delim_instruct = ''
eos_instruct = ' \noutput:'

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
aspects = []
n = df.shape[0]

for i in range(n):
    review = df['cleaned_reviews'][i]
    aspect_terms = ''
    for line in review.splitlines():
        sentences = sent_tokenize(line)
        for sentence in sentences:
            tokenized_text = tokenizer(bos_instruction + sentence + delim_instruct + eos_instruct, return_tensors = 'pt')
            output = model.generate(tokenized_text.input_ids)
            aspect = str(tokenizer.decode(output[0], skip_special_tokens = True))
            if aspect != 'noaspectterm: none':
                aspect_terms = aspect_terms + ', ' + aspect

    aspects.append(aspect_terms[2:])

In [ ]:
df['aspects'] = aspects
df

,title,stars,text,cleaned_reviews,aspects
0,Iro Iro Japanese Restaurant,5,We were so excited to eat so the photos are ta...,We were so excited to eat so the photos are ta...,"Tempura udon: neutral, fried tempura ebi: neut..."
1,Pura Brasa Singapore,5,Really great Spanish food at reasonable prices...,Really great Spanish food at reasonable prices...,"Spanish food: positive, prices: positive, gril..."
2,Sheik Jalani Restaurant,4,"Food range is amazing, food equality is slight...","Food range is amazing, food equality is slight...","Food range: positive, food equality: positive,..."
3,Black Society 黑社会,5,Nice ambience - the room was well decorated wi...,Nice ambience - the room was well decorated wi...,"ambience: positive, room: positive, Christmas ..."
4,BOMBAY DINING-NORTH INDIAN RESTAURANT,2,Ordered on Deliveroo and food arrived 50min la...,Ordered on Deliveroo and food arrived 50min la...,"food: neutral, Deliveroo: negative, delivery h..."
...,...,...,...,...,...
1063,Melben Seafood Restaurant - Depot Lane,2,Purchased a mongolian style beef rice with egg...,Purchased a mongolian style beef rice with egg...,"mongolian style beef rice with egg: neutral, c..."
1064,Tambi Restaurant,5,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,"Oxtail Bone Marrow Murtabak: positive, Kimchi:..."
1065,Ba Shu Sichuan Restaurant,5,Great tasting and authentic dishes here.\n\nWe...,Great tasting and authentic dishes here.\nWe o...,"dishes: positive, ,,, dishes: positive, : posi..."
1066,Mendon Bedok Mall,5,a small cozy nook beside Tori-Q that serves re...,a small cozy nook beside Tori-Q that serves re...,"nook beside Tori-Q: positive, dons: positive, ..."


In [ ]:
df.to_csv('drive/MyDrive/SC4021/test_absa_temp.csv')

# Post-processing

1.   Removing duplicates
2.   Removing items not in the form 'aspect: polarity'



In [ ]:
faulty = []

for i in df['aspects']:
    if i[-8:] != 'positive' and i[-8:] != 'negative' and i[-7:] != 'neutral' and i[-4:] != 'none':
        faulty.append(i)

print(len(faulty))
faulty

104


['coffee pork: neutral, prices: negative, sambal kang king:',
 'Hong Kong Cantonese-style Seafood Restaurant: positive, food quality: positive, quantity: positive, staff service: positive, Mr Adriano Lance: positive, public transport: positive, carpark lots: readily available',
 'sesame oil chicken: negative, wings part: negative, sambal sot',
 'food places: positive, lady boss: positive, service: positive, prices: negative, Okonomiyaki: positive, price: positive, instructions: provided, bonito flakes: positive, Potato Cheese Mentai: neutral, pricey: $12++, Garlic Fried Rice: neutral, Okonomiyaki: neutral, Potato Cheese Men',
 'omakase cuisine: neutral, plates: positive, mains: neutral, meal: neutral, oyster: positive, dish: neutral, fried potato ball: neutral, tofu egg: negative, chawamushi: neutral, sauce: negative, crab sauce dumplings: negative, fillings: negative, mains: sashimi don: positive, fried rice wagyu beef',
 'dishes in the menu: positive, atmosphere: negative, Price: neu

In [ ]:
for i in range(len(aspects)):
    aspect_terms = aspects[i].split(', ')
    aspect_terms = list(set(aspect_terms))
    aspect_terms = [item for item in aspect_terms if ":" in item and item.count(":") == 1 and item.split(":")[1].strip() in ['positive', 'negative', 'neutral']]
    aspects[i] = ', '.join(aspect_terms)

    if aspects[i] == '':
        aspects[i] = 'noaspectterm: none'

df['aspects'] = aspects
df

,title,stars,text,cleaned_reviews,aspects
0,Iro Iro Japanese Restaurant,5,We were so excited to eat so the photos are ta...,We were so excited to eat so the photos are ta...,"highball: neutral, udon: positive, flour: posi..."
1,Pura Brasa Singapore,5,Really great Spanish food at reasonable prices...,Really great Spanish food at reasonable prices...,"Spanish food: positive, grilled iberico pork s..."
2,Sheik Jalani Restaurant,4,"Food range is amazing, food equality is slight...","Food range is amazing, food equality is slight...","Food range: positive, patron: neutral, seats: ..."
3,Black Society 黑社会,5,Nice ambience - the room was well decorated wi...,Nice ambience - the room was well decorated wi...,"staffs: positive, service: positive, food item..."
4,BOMBAY DINING-NORTH INDIAN RESTAURANT,2,Ordered on Deliveroo and food arrived 50min la...,Ordered on Deliveroo and food arrived 50min la...,"wrapped in foil: negative, Deliveroo: negative..."
...,...,...,...,...,...
1063,Melben Seafood Restaurant - Depot Lane,2,Purchased a mongolian style beef rice with egg...,Purchased a mongolian style beef rice with egg...,"taste: negative, portion: negative, mongolian ..."
1064,Tambi Restaurant,5,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,Oxtail Bone Marrow Murtabak 8/10 (Kimchi was a...,"service: positive, Kimchi: positive, Oxtail Bo..."
1065,Ba Shu Sichuan Restaurant,5,Great tasting and authentic dishes here.\n\nWe...,Great tasting and authentic dishes here.\nWe o...,": positive, portion: negative, dishes: positive"
1066,Mendon Bedok Mall,5,a small cozy nook beside Tori-Q that serves re...,a small cozy nook beside Tori-Q that serves re...,"service charge: neutral, Prices: neutral, Japa..."


In [ ]:
faulty = []

for i in df['aspects']:
    if i[-8:] != 'positive' and i[-8:] != 'negative' and i[-7:] != 'neutral' and i[-4:] != 'none':
        faulty.append(i)

print(len(faulty))
faulty

0


[]

In [ ]:
df.to_csv('drive/MyDrive/SC4021/test_absa_final.csv')